In [ ]:
!apt update 
!apt install ffmpeg

!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install git+https://github.com/openai/whisper.git 
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8" 

import whisper
import cv2

# loading model
model = whisper.load_model('small')

# loading audio file
audio = whisper.load_audio('prompt.m4a')
# padding audio to 30 seconds
audio = whisper.pad_or_trim(audio)

# generating spectrogram
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# decoding
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# ready prompt!
prompt = result.text

# adding tips
prompt += 'Try to draw what you see in the picture'
print(prompt)

# generate the image corresponding to the prompt
img_path = "images/" + prompt + ".jpg"
img = cv2.imread(img_path)
cv2.imshow("Image Prompt", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# provide positive feedback to the child
print("Great job! You did it!")

import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    'CompVis/stable-diffusion-v1-4',
    revision='fp16',
    torcj_dtype=torch.float16,
    use_auth_token=True
)

pipe = pipe.to("cuda")

with torch.autocast('cuda'):
    image = pipe(prompt)['sample'][0]

import matplotlib.pyplot as plt

plt.imshow(image)
plt.title(prompt)
plt.axis('off')
plt.savefig('result.jpg')
plt.show()